In [ ]:
import json 

# Get list of cycle times required

sql = """
select station_name, lat, lng, nlc, icscode from tt_h.all_stations 
where in_london = true
"""

lon_stations = pd.read_sql(sql, conn)

lon_stations["j"] = 1

pcs = pd.DataFrame(pd.Series(["SW1H9AJ", "E145HP"]))
pcs.columns = ["destination"]
pcs["j"] = 1

lon_stations = lon_stations.merge(pcs)
lon_stations = lon_stations.drop("j", axis=1)

for r in lon_stations.iterrows():
    index = r[0]
    row = r[1]
    
    ci = get_cycle_info(origin_lat = row["lat"], origin_lng = row["lng"], dest_pc= row["destination"])
    
    lon_stations.loc[index, "cycle_minutes"] = ci["minutes"]
    lon_stations.loc[index, "cycle_miles"] = ci["miles"]

lon_stations.to_sql("cycle_times_in_london", engine, schema="tt_h", if_exists="replace", index=False)